## Import Required Libraries

In [9]:
import pandas as pd
import json
import os
from pathlib import Path
from typing import Dict, List, Any
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Configuration

In [10]:
# Path to audit records directory
AUDIT_RECORDS_DIR = Path(r"d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Streamlit app\audit_records")
OUTPUT_DIR = Path(r"d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data")

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Target workflows to filter
TARGET_WORKFLOWS = ['new_prompt_analysis', 'dataset_prompt_analysis']

print(f"Audit records directory: {AUDIT_RECORDS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Target workflows: {TARGET_WORKFLOWS}")

Audit records directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Streamlit app\audit_records
Output directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data
Target workflows: ['new_prompt_analysis', 'dataset_prompt_analysis']


## Load JSON Files

Recursively load all JSON files from the audit_records directory.

In [11]:
def load_audit_records(audit_dir: Path) -> List[Dict[str, Any]]:
    """
    Load all JSON files from audit_records directory recursively.
    Only loads files that start with 'audit'.
    
    Args:
        audit_dir: Path to the audit records directory
        
    Returns:
        List of parsed JSON records
    """
    records = []
    json_files = list(audit_dir.rglob('audit*.json'))
    
    print(f"Found {len(json_files)} JSON files starting with 'audit'")
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Add file path for reference
                data['_source_file'] = str(json_file)
                records.append(data)
        except json.JSONDecodeError as e:
            print(f"Error parsing {json_file}: {e}")
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    print(f"Successfully loaded {len(records)} records")
    return records

# Load all records
all_records = load_audit_records(AUDIT_RECORDS_DIR)
print(f"\nTotal records loaded: {len(all_records)}")

Found 84 JSON files starting with 'audit'
Successfully loaded 84 records

Total records loaded: 84


## Filter Records

Keep only records where workflow is either `new_prompt_analysis` or `dataset_prompt_analysis`.

In [12]:
def filter_analysis_workflows(records: List[Dict], target_workflows: List[str]) -> List[Dict]:
    """
    Filter records to keep only analysis workflows.
    
    Args:
        records: List of all audit records
        target_workflows: List of workflow names to keep
        
    Returns:
        Filtered list of records
    """
    filtered = []
    
    for record in records:
        workflow = record.get('workflow', '')
        if workflow in target_workflows:
            filtered.append(record)
    
    return filtered

# Filter records
analysis_records = filter_analysis_workflows(all_records, TARGET_WORKFLOWS)

print(f"Filtered records: {len(analysis_records)}")
print(f"\nWorkflow distribution:")
workflow_counts = {}
for record in analysis_records:
    workflow = record.get('workflow', 'unknown')
    workflow_counts[workflow] = workflow_counts.get(workflow, 0) + 1

for workflow, count in workflow_counts.items():
    print(f"  {workflow}: {count}")

Filtered records: 84

Workflow distribution:
  dataset_prompt_analysis: 42
  new_prompt_analysis: 42


## Extract Workflow Data

Group by workflow_id and extract all required fields into a structured format.

In [13]:
def safe_get(dictionary: Dict, *keys, default=None):
    """
    Safely get nested dictionary values.
    
    Args:
        dictionary: Input dictionary
        *keys: Keys to traverse
        default: Default value if key doesn't exist
        
    Returns:
        Value at the nested key, or default
    """
    current = dictionary
    for key in keys:
        if isinstance(current, dict):
            current = current.get(key, default)
        else:
            return default
    return current if current is not None else default


def extract_cwe_list(vulnerabilities: list) -> list:
    """
    Extract unique CWE IDs from vulnerability list.
    
    Args:
        vulnerabilities: List of vulnerability dictionaries
        
    Returns:
        Sorted list of unique CWE IDs
    """
    if not vulnerabilities:
        return []
    
    cwe_ids = set()
    for vuln in vulnerabilities:
        cwe_id = vuln.get('cwe_id', vuln.get('test_id', ''))
        if cwe_id:
            # Normalize CWE ID (remove 'CWE-' prefix if present)
            cwe_id = str(cwe_id).replace('CWE-', '').strip()
            cwe_ids.add(cwe_id)
    
    return sorted(list(cwe_ids))


def count_fix_providers_by_cwe(record: Dict) -> Dict[str, Any]:
    """
    Count fixes by provider (LLM vs custom rule) and track which unique CWEs were fixed by each.
    
    Args:
        record: Audit record
        
    Returns:
        Dictionary with counts for each fix provider and CWE lists
    """
    llm_cwes = set()
    rule_cwes = set()
    unknown_cwes = set()
    
    # Check initial patch result
    initial_changes = safe_get(record, 'initial_patch_result', 'changes', default=[])
    for change in initial_changes:
        patch_method = change.get('patch_method', '').lower()
        cwe_id = change.get('cwe_id', '')
        if cwe_id:
            cwe_id = str(cwe_id).replace('CWE-', '').strip()
            if 'llm' in patch_method:
                llm_cwes.add(cwe_id)
            elif 'rule' in patch_method:
                rule_cwes.add(cwe_id)
            else:
                unknown_cwes.add(cwe_id)
    
    # Check patch iterations
    patch_iterations = safe_get(record, 'patch_iterations', default=[])
    for iteration in patch_iterations:
        changes = iteration.get('changes', [])
        for change in changes:
            patch_method = change.get('patch_method', '').lower()
            cwe_id = change.get('cwe_id', '')
            if cwe_id:
                cwe_id = str(cwe_id).replace('CWE-', '').strip()
                if 'llm' in patch_method:
                    llm_cwes.add(cwe_id)
                elif 'rule' in patch_method:
                    rule_cwes.add(cwe_id)
                else:
                    unknown_cwes.add(cwe_id)
    
    return {
        'llm': len(llm_cwes),
        'rule_based': len(rule_cwes),
        'unknown': len(unknown_cwes),
        'llm_cwes': sorted(list(llm_cwes)),
        'rule_cwes': sorted(list(rule_cwes)),
        'unknown_cwes': sorted(list(unknown_cwes))
    }


def extract_workflow_data(record: Dict) -> Dict[str, Any]:
    """
    Extract all required fields from a single workflow record.
    
    Args:
        record: Audit record (can be flat or nested format)
        
    Returns:
        Dictionary with extracted workflow data
    """
    # Check if this is already a flattened audit record
    # (has fields like 'initial_detection_bandit_count' at top level)
    is_flat = 'initial_detection_bandit_count' in record
    
    if is_flat:
        # Data is already in the correct format, just return it
        workflow = record.get('workflow', '')
        return {
            'workflow_id': record.get('workflow_id', ''),
            'workflow': workflow,
            'timestamp': record.get('timestamp', ''),
            'file': record.get('file', ''),
            'source_file': record.get('source_file', ''),
            'prompt_type': 'Manual' if workflow == 'new_prompt_analysis' else 'SecurityEval',
            'prompt': record.get('original_content', ''),
            'llm_response': record.get('llm_response', ''),
            'vulnerabilities_found': record.get('vulnerabilities_found', 0),
            'total_vulnerabilities_identified': record.get('total_vulnerabilities_identified', 0),
            'total_vulnerabilities_fixed': record.get('total_vulnerabilities_fixed', 0),
            'total_vulnerabilities_remaining': record.get('total_vulnerabilities_remaining', 0),
            'initial_detection_bandit_count': record.get('initial_detection_bandit_count', 0),
            'initial_detection_bandit_cwes': record.get('initial_detection_bandit_cwes', []),
            'initial_detection_semgrep_count': record.get('initial_detection_semgrep_count', 0),
            'initial_detection_semgrep_cwes': record.get('initial_detection_semgrep_cwes', []),
            'initial_detection_ast_count': record.get('initial_detection_ast_count', 0),
            'initial_detection_ast_cwes': record.get('initial_detection_ast_cwes', []),
            'iteration_detection_bandit_count': record.get('iteration_detection_bandit_count', 0),
            'iteration_detection_bandit_cwes': record.get('iteration_detection_bandit_cwes', []),
            'iteration_detection_semgrep_count': record.get('iteration_detection_semgrep_count', 0),
            'iteration_detection_semgrep_cwes': record.get('iteration_detection_semgrep_cwes', []),
            'iteration_detection_ast_count': record.get('iteration_detection_ast_count', 0),
            'iteration_detection_ast_cwes': record.get('iteration_detection_ast_cwes', []),
            'iterations_count': record.get('iterations_count', 0),
            'fixed_cwes': record.get('fixed_cwes', []),
            'remaining_cwes': record.get('remaining_cwes', []),
            'fix_provider_llm': record.get('fix_provider_llm', 0),
            'fix_provider_rule_based': record.get('fix_provider_rule_based', 0),
            'fix_provider_unknown': record.get('fix_provider_unknown', 0),
            'fix_provider_llm_cwes': record.get('fix_provider_llm_cwes', []),
            'fix_provider_rule_cwes': record.get('fix_provider_rule_cwes', []),
            'fix_provider_unknown_cwes': record.get('fix_provider_unknown_cwes', []),
        }
    
    # Otherwise, handle nested format (original workflow records)
    # Basic information
    workflow = record.get('workflow', '')
    data = {
        'workflow_id': record.get('workflow_id', ''),
        'workflow': workflow,
        'timestamp': record.get('timestamp', ''),
        'file': record.get('file', ''),
        'source_file': record.get('_source_file', ''),
        'prompt_type': 'Manual' if workflow == 'new_prompt_analysis' else 'SecurityEval',
    }
    
    # Prompt and response
    data['prompt'] = record.get('content', record.get('original_content', ''))
    data['llm_response'] = record.get('response', '')
    
    # Vulnerability counts (these include duplicates)
    data['vulnerabilities_found'] = record.get('vulnerabilities_found', 0)
    data['total_vulnerabilities_identified'] = record.get('total_vulnerabilities_identified', 0)
    data['total_vulnerabilities_fixed'] = record.get('total_vulnerabilities_fixed', 0)
    data['total_vulnerabilities_remaining'] = record.get('total_vulnerabilities_remaining', 0)
    
    # Initial detection by tool
    initial_run = record.get('initial_run_by_tool', {})
    
    # Bandit
    bandit_vulns = safe_get(initial_run, 'bandit', 'identified_vulnerabilities', default=[])
    data['initial_detection_bandit_count'] = safe_get(initial_run, 'bandit', 'count', default=0)
    data['initial_detection_bandit_cwes'] = extract_cwe_list(bandit_vulns)
    
    # Semgrep
    semgrep_vulns = safe_get(initial_run, 'semgrep', 'identified_vulnerabilities', default=[])
    data['initial_detection_semgrep_count'] = safe_get(initial_run, 'semgrep', 'count', default=0)
    data['initial_detection_semgrep_cwes'] = extract_cwe_list(semgrep_vulns)
    
    # AST (custom_detector)
    ast_vulns = safe_get(initial_run, 'custom_detector', 'identified_vulnerabilities', default=[])
    data['initial_detection_ast_count'] = safe_get(initial_run, 'custom_detector', 'count', default=0)
    data['initial_detection_ast_cwes'] = extract_cwe_list(ast_vulns)
    
    # Iteration detection by tool
    iterations_by_tool = record.get('iterations_by_tool', {})
    
    # Bandit iterations
    bandit_iter_vulns = safe_get(iterations_by_tool, 'bandit', 'identified_vulnerabilities', default=[])
    data['iteration_detection_bandit_count'] = safe_get(iterations_by_tool, 'bandit', 'total_across_all_iterations', default=0)
    data['iteration_detection_bandit_cwes'] = extract_cwe_list(bandit_iter_vulns)
    
    # Semgrep iterations
    semgrep_iter_vulns = safe_get(iterations_by_tool, 'semgrep', 'identified_vulnerabilities', default=[])
    data['iteration_detection_semgrep_count'] = safe_get(iterations_by_tool, 'semgrep', 'total_across_all_iterations', default=0)
    data['iteration_detection_semgrep_cwes'] = extract_cwe_list(semgrep_iter_vulns)
    
    # AST iterations
    ast_iter_vulns = safe_get(iterations_by_tool, 'custom_detector', 'identified_vulnerabilities', default=[])
    data['iteration_detection_ast_count'] = safe_get(iterations_by_tool, 'custom_detector', 'total_across_all_iterations', default=0)
    data['iteration_detection_ast_cwes'] = extract_cwe_list(ast_iter_vulns)
    
    # Iterations count
    data['iterations_count'] = len(record.get('patch_iterations', []))
    
    # Fixed and remaining CWEs (these are unique)
    data['fixed_cwes'] = record.get('fixed_cwe_ids', [])
    data['remaining_cwes'] = record.get('non_fixed_cwe_ids', [])
    
    # Fix provider counts by unique CWE
    fix_providers = count_fix_providers_by_cwe(record)
    data['fix_provider_llm'] = fix_providers['llm']
    data['fix_provider_rule_based'] = fix_providers['rule_based']
    data['fix_provider_unknown'] = fix_providers['unknown']
    data['fix_provider_llm_cwes'] = fix_providers['llm_cwes']
    data['fix_provider_rule_cwes'] = fix_providers['rule_cwes']
    data['fix_provider_unknown_cwes'] = fix_providers['unknown_cwes']
    
    return data

print("Data extraction functions defined successfully!")

Data extraction functions defined successfully!


## Build DataFrame

Process all filtered records and create a DataFrame with one row per workflow.

In [14]:
# Extract data from all analysis records
workflow_data = []

for record in analysis_records:
    try:
        data = extract_workflow_data(record)
        workflow_data.append(data)
    except Exception as e:
        print(f"Error processing record {record.get('workflow_id', 'unknown')}: {e}")

# Create DataFrame
df = pd.DataFrame(workflow_data)

print(f"DataFrame created with {len(df)} rows and {len(df.columns)} columns")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

DataFrame created with 84 rows and 33 columns

DataFrame shape: (84, 33)

Column names:
  1. workflow_id
  2. workflow
  3. timestamp
  4. file
  5. source_file
  6. prompt_type
  7. prompt
  8. llm_response
  9. vulnerabilities_found
  10. total_vulnerabilities_identified
  11. total_vulnerabilities_fixed
  12. total_vulnerabilities_remaining
  13. initial_detection_bandit_count
  14. initial_detection_bandit_cwes
  15. initial_detection_semgrep_count
  16. initial_detection_semgrep_cwes
  17. initial_detection_ast_count
  18. initial_detection_ast_cwes
  19. iteration_detection_bandit_count
  20. iteration_detection_bandit_cwes
  21. iteration_detection_semgrep_count
  22. iteration_detection_semgrep_cwes
  23. iteration_detection_ast_count
  24. iteration_detection_ast_cwes
  25. iterations_count
  26. fixed_cwes
  27. remaining_cwes
  28. fix_provider_llm
  29. fix_provider_rule_based
  30. fix_provider_unknown
  31. fix_provider_llm_cwes
  32. fix_provider_rule_cwes
  33. fix_prov

In [15]:
def clean_and_standardize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and standardize the DataFrame.
    
    Args:
        df: Input DataFrame
        
    Returns:
        Cleaned DataFrame
    """
    df_clean = df.copy()
    
    # Ensure list columns are actual lists (not strings or NaN)
    list_columns = [
        'initial_detection_bandit_cwes',
        'initial_detection_semgrep_cwes',
        'initial_detection_ast_cwes',
        'iteration_detection_bandit_cwes',
        'iteration_detection_semgrep_cwes',
        'iteration_detection_ast_cwes',
        'fixed_cwes',
        'remaining_cwes',
        'fix_provider_llm_cwes',
        'fix_provider_rule_cwes',
        'fix_provider_unknown_cwes'
    ]
    
    for col in list_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(
                lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [])
            )
    
    # Ensure count columns are integers
    count_columns = [
        'vulnerabilities_found',
        'total_vulnerabilities_identified',
        'total_vulnerabilities_fixed',
        'total_vulnerabilities_remaining',
        'initial_detection_bandit_count',
        'initial_detection_semgrep_count',
        'initial_detection_ast_count',
        'iteration_detection_bandit_count',
        'iteration_detection_semgrep_count',
        'iteration_detection_ast_count',
        'iterations_count',
        'fix_provider_llm',
        'fix_provider_rule_based',
        'fix_provider_unknown'
    ]
    
    for col in count_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype(int)
    
    # Handle missing string values
    string_columns = ['workflow_id', 'workflow', 'timestamp', 'file', 'source_file', 'prompt', 'llm_response', 'prompt_type']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    # Convert timestamp to datetime
    if 'timestamp' in df_clean.columns:
        df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], errors='coerce')
    
    # Add computed columns for unique CWE counts
    if 'fixed_cwes' in df_clean.columns:
        df_clean['unique_cwes_fixed'] = df_clean['fixed_cwes'].apply(len)
    
    if 'remaining_cwes' in df_clean.columns:
        df_clean['unique_cwes_remaining'] = df_clean['remaining_cwes'].apply(len)
    
    # Calculate total unique CWEs identified (fixed + remaining)
    if 'fixed_cwes' in df_clean.columns and 'remaining_cwes' in df_clean.columns:
        df_clean['unique_cwes_identified'] = df_clean.apply(
            lambda row: len(set(row['fixed_cwes']) | set(row['remaining_cwes'])),
            axis=1
        )
    
    return df_clean

# Apply cleaning
df = clean_and_standardize_dataframe(df)

print("DataFrame cleaned and standardized successfully!")
print(f"\nUpdated shape: {df.shape}")
print(f"\nNew computed columns added:")
print("  - unique_cwes_identified: Count of unique CWEs found")
print("  - unique_cwes_fixed: Count of unique CWEs fixed")
print("  - unique_cwes_remaining: Count of unique CWEs remaining")

DataFrame cleaned and standardized successfully!

Updated shape: (84, 36)

New computed columns added:
  - unique_cwes_identified: Count of unique CWEs found
  - unique_cwes_fixed: Count of unique CWEs fixed
  - unique_cwes_remaining: Count of unique CWEs remaining


## Data Cleaning and Standardization

Clean and standardize the DataFrame to ensure consistent data types and handle missing values.

In [16]:
# Export complete dataset (used by downstream notebooks)
evaluation_base_path = OUTPUT_DIR / 'evaluation_base.csv'
df.to_csv(evaluation_base_path, index=False)
print(f"✓ Complete dataset saved to: {evaluation_base_path}")
print(f"  Total workflows: {len(df)}")


✓ Complete dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\evaluation_base.csv
  Total workflows: 84
